<a href="https://colab.research.google.com/github/FaiazS/LLaMA-SQL-3.1-Fine-Tuning-Meta-s-LLaMA-3.1-for-Natural-Language-to-SQL-Generation-using-QLoRA/blob/main/Fine_Tuning_LLM(Text_to_SQL).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Installing the required Libraries

!pip install bitsandbytes

!pip install accelerate

!pip install xformers==0.0.29.post3

!pip install peft

!pip install trl

!pip install triton

!pip install cut_cross_entropy

!pip install unsloth_zoo

!pip install sentencepiece

!pip install protobuf

!pip install datasets

!pip install huggingface_hub

!pip install hf_transfer

!pip install unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 43.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 6.8 MB/s eta 0:00:00


In [ ]:
#Setting up and Configuring the Large Language Model and Applying 4 bit Quantization Low Rank Adaptation(Q-LoRA)

from unsloth import FastLanguageModel

import torch

device_map = {"": 0}

llm_model, llm_model_tokenizer = FastLanguageModel.from_pretrained(

                                              model_name = "unsloth/Meta-Llama-3.1-8B",

                                              max_seq_length = 2048,

                                              load_in_4bit =  True,

                                              device_map = device_map
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [ ]:
un_finetuned_llm_model = llm_model

In [ ]:
#Low Rank Adaptation(LoRA) PEFT Configuration for the LLM

llm_model = FastLanguageModel.get_peft_model(


                                             llm_model,

                                             r = 16,

                                             target_modules = ["q_proj", "k_proj","v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],

                                             lora_alpha = 16,

                                             lora_dropout = 0,

                                             bias = "none",

                                             use_gradient_checkpointing = "unsloth",

                                             random_state = 3407,

                                             use_rslora = False,

                                             loftq_config = None



)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
#Formatting the Dataset into 'Alpaca Format' SPECIFICALLY as we are using Meta's Llama LLM Model before feeding the Dataset to the Pre-finetuned Llama LLM Model for Fine-tuning

alpaca_format = """Below is an instruction that provides a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:

Company database: {}

### Input:

SQL Prompt: {}

### Response:

SQL {}

### Explanation:

{}

"""

EOS_TOKEN = llm_model_tokenizer.eos_token

def format_inputs_to_prompts(examples):

  company_database = examples["sql_context"]

  prompts = examples["sql_prompt"]

  sqls= examples["sql"]

  explanations = examples["sql_explanation"]

  input_texts = []

  for company_database, prompt, sql, explanation in zip(company_database, prompts, sqls, explanations):

    input_text = alpaca_format.format(company_database, prompt, sql, explanation) + EOS_TOKEN

    input_texts.append(input_text)

  return {"input_text" : input_texts}

pass

In [ ]:
#Loading the Dataset post Formatting

from datasets import load_dataset

dataset = load_dataset("gretelai/synthetic_text_to_sql")

dataset = dataset['train']

dataset = dataset.map(format_inputs_to_prompts, batched = True)

README.md:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

(…)nthetic_text_to_sql_train.snappy.parquet:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

(…)ynthetic_text_to_sql_test.snappy.parquet:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
dataset['input_text'][0]

"Below is an instruction that provides a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCompany database: CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');\n\n### Input:\n\nSQL Prompt: What is the total volume of timber sold by each salesperson, sorted by salesperson?\n\n### Response:\n\nSQL SELECT salesperson_id, name, SUM(volume) as total_volume FROM timber_sales JOIN salesperson ON timber_sales.salesperson_id = salesperson.salesperson_id GROUP BY salesperson_id, name ORDER BY total_volume DESC;\n\n### Explanatio

In [ ]:
dataset.shape

(100000, 12)

In [ ]:
#Initializing the Fine tuning process(Begin Training the Meta's Llama LLM Model)

from trl import SFTTrainer

from transformers import TrainingArguments

from unsloth import is_bfloat16_supported

llm_model_fine_tuner = SFTTrainer(

                               model = llm_model,

                               tokenizer = llm_model_tokenizer,

                               train_dataset = dataset,

                               dataset_text_field = "input_text",

                               max_seq_length = 2048,

                               dataset_num_proc = 2,

                               packing = False,

                               args = TrainingArguments(

                                                        per_device_train_batch_size = 2,

                                                        gradient_accumulation_steps = 4,

                                                        warmup_steps = 5,

                                                        num_train_epochs = 1,

                                                        max_steps = 60,

                                                        learning_rate = 2e-4,

                                                        fp16 = not is_bfloat16_supported(),

                                                        bf16 = is_bfloat16_supported(),

                                                        logging_steps = 1,

                                                        optim = "adamw_8bit",

                                                        weight_decay = 0.01,

                                                        lr_scheduler_type = "linear",

                                                        seed = 3407,

                                                        output_dir = "outputs",

                                                        report_to = "none"








                               )
)


llm_model_fine_tuner.train()

Unsloth: Tokenizing ["input_text"] (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.653700
2,1.647000
3,1.611000
4,1.506500
5,1.301000
6,1.155900
7,1.053900
8,0.840100
9,0.796000
10,0.820100


TrainOutput(global_step=60, training_loss=0.6466894701123238, metrics={'train_runtime': 440.2176, 'train_samples_per_second': 1.09, 'train_steps_per_second': 0.136, 'total_flos': 5626008754470912.0, 'train_loss': 0.6466894701123238})

In [ ]:
#Inference/Evaluation and Comparision between Fine-tuned and not Fine-tuned LLM Models post Fine-tuning

FastLanguageModel.for_inference(un_finetuned_llm_model)

text_inputs = llm_model_tokenizer(
    [
                              alpaca_format.format(

                                                  "Company database: CREATE TABLE Patients(PatientID INT, Name TEXT, Age INT, Diagnosis TEXT); INSERT INTO Patients(PatientID, Name, Age, Diagnosis) VALUES (1,'Alice Johnson',30,'Diabetes'),(2, 'Bob Williams',45,'Hypertension')(3, 'Charlie Brown',30, 'Diabetes');",

                                                  "SQL Prompt: How many patients have each diagnosis?", #Input

                                                  "", #Response

                                                  ""  #Explanation

                            )

    ] , return_tensors = "pt"

)

text_inputs = text_inputs.to("cuda")

outputs = un_finetuned_llm_model.generate(**text_inputs, max_new_tokens = 128, use_cache = True)

llm_response1 = llm_model_tokenizer.batch_decode(outputs)

FastLanguageModel.for_inference(llm_model)

text_inputs = llm_model_tokenizer(

                            [

                            alpaca_format.format(

                                                 "Company database: CREATE TABLE Orders(Order ID INT, Customer ID INT, Order date DATE, Total Amount FLOAT); INSERT INTO Orders(Order ID, Customer ID, Order date, Total Amount) VALUES (1,101,'2024-1-15' 250.75 ),(2,102,'2024-1-16',400.00),(3,103,'2024-1-17',350.20)",

                                                 "SQL Prompt: What is the total amount spent by each customer?", #Input

                                                 "",  #Response

                                                 ""  #Explanation
                            )

                            ], return_tensors = "pt"

)

text_inputs = text_inputs.to("cuda")

outputs = llm_model.generate(**text_inputs, max_new_tokens = 128, use_cache = True)

llm_response2 = llm_model_tokenizer.batch_decode(outputs)

FastLanguageModel.for_inference(llm_model)

text_inputs = llm_model_tokenizer(
                        [
                            alpaca_format.format(

                                                 "Company database:CREATE TABLE Patients(PatientID INT, Name TEXT, Age INT, Diagnosis TEXT); INSERT INTO Patients(PatientID, Name, Age, Diagnosis) VALUES (1,'Alice Johnson',30,'Diabetes'),(2, 'Bob Williams',45,'Hypertension')(3, 'Charlie Brown',30, 'Diabetes');",

                                                 "SQL Prompt: How many patients have each diagnosis?", #Input

                                                 "",  #Response

                                                 ""  #Explanation


                            )

                        ], return_tensors = "pt"
)

text_inputs = text_inputs.to("cuda")

outputs = llm_model.generate(**text_inputs, max_new_tokens = 128, use_cache = True)

llm_response3 = llm_model_tokenizer.batch_decode(outputs)

FastLanguageModel.for_inference(llm_model)

text_inputs = llm_model_tokenizer(

                            [

                            alpaca_format.format(

                                                 "Company database: CREATE TABLE Transaction(Transaction ID INT, Account ID INT, Type TEXT, Amount FLOAT); INSERT INTO(Transaction ID, Account ID, Type, Amount) VALUES (0001, 0123, Savings, 250.00),(0002, 0124, Current,5700.00),(0002, 0124, Savings, 520.00)",

                                                 "SQL Prompt:", #Input

                                                 "", #Response

                                                 "" #Explanation


                            )

                            ], return_tensors = "pt"

)

text_inputs = text_inputs.to("cuda")

outputs = llm_model.generate(**text_inputs, max_new_tokens = 128, use_cache = True)

llm_response4 = llm_model_tokenizer.batch_decode(outputs)

In [ ]:
print(llm_response1)

["<|begin_of_text|>Below is an instruction that provides a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCompany database: Company database: CREATE TABLE Patients(PatientID INT, Name TEXT, Age INT, Diagnosis TEXT); INSERT INTO Patients(PatientID, Name, Age, Diagnosis) VALUES (1,'Alice Johnson',30,'Diabetes'),(2, 'Bob Williams',45,'Hypertension')(3, 'Charlie Brown',30, 'Diabetes');\n\n### Input:\n\nSQL Prompt: SQL Prompt: How many patients have each diagnosis?\n\n### Response:\n\nSQL \n\n### Explanation:\n\n\n\n<|end_of_text|>"]


In [ ]:
print(llm_response2)

["<|begin_of_text|>Below is an instruction that provides a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCompany database: Company database: CREATE TABLE Orders(Order ID INT, Customer ID INT, Order date DATE, Total Amount FLOAT); INSERT INTO Orders(Order ID, Customer ID, Order date, Total Amount) VALUES (1,101,'2024-1-15' 250.75 ),(2,102,'2024-1-16',400.00),(3,103,'2024-1-17',350.20)\n\n### Input:\n\nSQL Prompt: SQL Prompt: What is the total amount spent by each customer?\n\n### Response:\n\nSQL \n\n### Explanation:\n\n\n\nSQL Prompt: SQL Prompt: What is the average order amount for each customer?\n\n### Response:\n\nSQL SELECT CustomerID, AVG(TotalAmount) FROM Orders GROUP BY CustomerID;\n\n### Explanation:\n\nThis query calculates the average order amount for each customer by grouping the data by CustomerID and calculating the average of the TotalAmount column.\n\nSQL Prompt: SQL Prompt: Wha

In [ ]:
print(llm_response3)

["<|begin_of_text|>Below is an instruction that provides a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCompany database: Company database:CREATE TABLE Patients(PatientID INT, Name TEXT, Age INT, Diagnosis TEXT); INSERT INTO Patients(PatientID, Name, Age, Diagnosis) VALUES (1,'Alice Johnson',30,'Diabetes'),(2, 'Bob Williams',45,'Hypertension')(3, 'Charlie Brown',30, 'Diabetes');\n\n### Input:\n\nSQL Prompt: SQL Prompt: How many patients have each diagnosis?\n\n### Response:\n\nSQL \n\n### Explanation:\n\n\n\n<|end_of_text|>"]


In [ ]:
print(llm_response4)

['<|begin_of_text|>Below is an instruction that provides a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n\nCompany database: Company database: CREATE TABLE Transaction(Transaction ID INT, Account ID INT, Type TEXT, Amount FLOAT); INSERT INTO(Transaction ID, Account ID, Type, Amount) VALUES (0001, 0123, Savings, 250.00),(0002, 0124, Current,5700.00),(0002, 0124, Savings, 520.00)\n\n### Input:\n\nSQL Prompt: SQL Prompt:\n\n### Response:\n\nSQL \n\n### Explanation:\n\n\n\n<|end_of_text|>']


In [ ]:
import re

output_text1 = llm_response1[0]

output_text2 = llm_response2[0]

output_text3 = llm_response3[0]

output_text4 = llm_response4[0]

#Using Regular Expression to extract the SQL Query

match = re.search(r"SQL:\s*(SELECT .*?;)", output_text1, re.DOTALL)

if match:

  sql_query = match.group(1).strip()

  print("Extracted SQL Query:", sql_query)

else:

  print("No SQL Query found in the response!")

No SQL Query found in the response!
